In [1]:
import pymarc as pm
import pandas as pd
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'src/')
from marc_get_info import *

#### Statistiky

In [2]:
n = 0
english = 0
czech = 0
with open('marc/albatros_literatura_mladez.mrc', 'rb') as f:
   reader = pm.MARCReader(f)
   for record in reader:
    n += 1
    for field in record.get_fields('041'):
        for subfield in field.get_subfields('a'):
            if subfield == 'eng': 
                english += 1
                for f in record.get_fields('245'): print(f)
            if subfield == 'cze': czech += 1
print(f'Number of records: {n}')    
print(f'Number of english records: {english}')   
print(f'Number of czech records: {czech}')    

=245  10$aLittle mole and Christmas /$c[illustrated by] Zdeněk Miler ; [written by] Hana Doskočilová ; [translated from the Czech by Mike and Tereza Baugh, with the assistance of Terry Baugh]
=245  10$aFlirt but don't fall in love /$cJulia Rossová ; [přeložil Tomáš Bíla]
=245  10$aLittle mole and mother /$cconcept and illustrations by Zdeněk Miler ; written by Hana Doskočilová ; [translated by Hana Parkánová-Whitton from the Czech original ...]
=245  10$aLittle mole in spring /$c[concept and illustrations] Zdeněk Miler, Kateřina Miler ; [written by] Hana Doskočilová ; [translated from the Czech by Mike and Tereza Baugh]
=245  10$aLittle mole in summer /$c[concept and illustrations] Zdeněk Miler, Kateřina Miler ; [written by] Hana Doskočilová ; [translated from the Czech by Mike and Tereza Baugh]
=245  10$aLittle mole and the snowman /$cconcept and illustrations Zdeněk Miler ; text Hana Doskočilová ; [translated from Czech by Mike and Tereza Baugh]
=245  10$aLittle mole and his little c

#### Matching titles

In [3]:
podklady_albatros = pd.read_excel('excel tables/Přehled Albatros a Scraping.xlsx')
book_names = list(podklady_albatros['Book Name'].apply(lambda x: x.lower()))
found_books = set()
n = 0
ids = []
OUT = 'marc/albatros_matched_recordes.mrc'
with open(OUT , 'wb') as writer:
    with open('marc/albatros_literatura_mladez.mrc', 'rb') as f:
        reader = pm.MARCReader(f)
        for record in reader:
            title = record.title
            tit_orig = 'b'
            tit = 'a'
            for field in record.get_fields('245'):
                for subfield in field.get_subfields('a'):
                    tit = subfield.strip(' ./:=')
            for field in record.get_fields('240'):
                for subfield in field.get_subfields('a'):
                    tit_orig = subfield.strip(' ./:=')   

            for i in [title.lower(),  tit.lower(), tit_orig.lower()]:
                if i in book_names:            
                    n+= 1
                    found_books.add(title.lower())
                    writer.write(record.as_marc())
                    ids.append(book_names.index(i))
                    print(title)
                    break
print(f'{len(found_books)}')            


Little mole and Christmas /
Little mole and mother /
Little mole in spring /
Little mole in summer /
Little mole and his little car /
Maxipes Fík /
Little mole in fall /
Kluk a pes /
Little mole in winter /
How little mole got his trousers /
Mach a Šebestová ve škole /
Little mole and mother /
Otazníky detektiva Štiky /
Little mole in fall /
Little mole in winter /
Little mole in summer /
Little mole in spring /
Little mole and Christmas /
Robinson Crusoe /
Lentilka pro dědu Edu /
How little mole got his trousers /
Robinson Crusoe /
Hrátky pro šikovné ruce /
Kosprd a Telecí : příběh ze školky /
Cinderella = Popelka /
How little mole cured little mouse /
Na orlích křídlech /
Jak velbloud potkal ťavu : česko-slovenský obrázkový slovník zákeřných slov pro děti /
Zajatci Minecraftu /
Mach a Šebestová na prázdninách /
Robinson Crusoe /
Příhody maxipsa Fíka /
Traktor /
Mach a Šebestová na cestách /
Robinson Crusoe.
Řvi potichu, brácho /
Štaflík a Špagetka /
Kluk a pes /
Maxipes Fík /
Zachrán

In [6]:

df_dict = {'ID': [], 'Book Name':[], 'Book Name Original': [], 'Author NK':[], 'Illustrator NK': [], 'Pages': [], 'Publication Year':[], 'Original Language': [], 'Text Language': [],  'Albatros NK':[]}

#df_dict = {'Book Name': [], 'Book Name Original': [], 'Author':[], 'Illustrator': [], 'Pages': [], '655': []}

with open('marc/albatros_matched_recordes.mrc', 'rb') as f:
    reader = pm.MARCReader(f)
    for id, record in zip(ids, reader):

        df_dict['ID'].append(id)

        language_orig, language_text = get_041(record)
        df_dict['Original Language'].append(language_orig)
        df_dict['Text Language'].append(language_text)

        title = get_title(record)
        df_dict['Book Name'].append(title)

        title_orig = get_original_title(record)
        df_dict['Book Name Original'].append(title_orig)

        author, illustrator = get_author_illustrator(record)
        df_dict['Author NK'].append(author)
        df_dict['Illustrator NK'].append(illustrator)

        year, pages = get_year_page(record)
        df_dict['Publication Year'].append(year)
        df_dict['Pages'].append(pages)
        df_dict['Albatros NK'].append(True)
df_b4u = pd.DataFrame(df_dict) 
df_b4u = df_b4u.drop_duplicates(subset=['Book Name', 'Book Name Original', 'Pages', 'Publication Year', 'Original Language', 'Text Language'])
df_b4u = df_b4u.set_index('ID')

    

In [7]:
podklady_albatros = pd.read_excel('excel tables/Přehled Albatros a Scraping.xlsx')
podklady_albatros.Scraping = podklady_albatros.Scraping.apply(lambda x: True if x == 1 else None)
podklady_albatros.Albatros = podklady_albatros.Albatros.apply(lambda x: True if x == 1 else None) 
df = pd.merge(podklady_albatros, df_b4u, how="outer", left_index=True, right_index=True)